<a href="https://colab.research.google.com/github/Shreyanshy53/Alarm-App/blob/main/BugDetectionProject/BugDetectionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
import os

data_path = "/content/drive/MyDrive/BugDetectionProject"

print("Files in BugDetectionProject:", os.listdir(data_path))


Files in BugDetectionProject: ['codenetpy_train.json', 'codenetpy_test.json', 'BugDetectionProject.ipynb']


dataset Load

In [3]:
import json

train_file = "/content/drive/MyDrive/BugDetectionProject/codenetpy_train.json" #change the path
test_file = "/content/drive/MyDrive/BugDetectionProject/codenetpy_test.json" #please change the path of dataset

def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)


    if isinstance(data, dict) and "data" in data:
        return data["data"]

    return data


train_data = load_json(train_file)
test_data = load_json(test_file)
print("Total training samples:", len(train_data))
print("Total testing samples:", len(test_data))
print("Example sample:", train_data[0])


Total training samples: 43673
Total testing samples: 10919
Example sample: {'original_src': 'l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]', 'changed_src': 'l=[]\nwhile True:\n    try:\n        l.append(input())\n    except EOFError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]', 'problem_id': 'p00001', 'original_id': 's196059089', 'changed_id': 's508355022', 'language': 'Python', 'filename_ext': 'py', 'original_status': 'Runtime Error', 'returncode': 1, 'error_class': 'SyntaxError', 'error_class_extra': "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?", 'error': '  File "/home/alex/Documents/research/bug-detection/../input/Project_CodeNet/data/p00001/Python/s196059089.py", line 9\n    print l[i]\n          ^\nSyntaxError: Missing parentheses in call to \'print\'. Did you mean print(l[i])?\n', 'output': ''}


In [4]:
import json
import tokenize
from io import BytesIO
import ast

In [5]:

def python_tokenizer(code):
    tokens = []
    try:
        token_stream = tokenize.tokenize(BytesIO(code.encode('utf-8')).readline)
        for token in token_stream:
            if token.type != tokenize.ENCODING:
                tokens.append(token.string)
    except Exception as e:
        print("Tokenization Error:", e)
    return tokens

In [6]:

def extract_ast_tokens(code):
    try:
        tree = ast.parse(code)
        tokens = [node.__class__.__name__ for node in ast.walk(tree)]
        return tokens
    except Exception as e:
        print("AST Parsing Error:", e)
        return []

In [7]:
import json

train_path = "/content/drive/MyDrive/BugDetectionProject/codenetpy_train.json"
test_path = "/content/drive/MyDrive/BugDetectionProject/codenetpy_test.json"
with open(train_path, "r") as f:
    train_data = json.load(f)
with open(test_path, "r") as f:
    test_data = json.load(f)
print("Train Data Keys:", train_data.keys())
print("Test Data Keys:", test_data.keys())

first_sample = list(train_data["data"])[0]
print("First Sample Keys:", first_sample.keys())

Train Data Keys: dict_keys(['data'])
Test Data Keys: dict_keys(['data'])
First Sample Keys: dict_keys(['original_src', 'changed_src', 'problem_id', 'original_id', 'changed_id', 'language', 'filename_ext', 'original_status', 'returncode', 'error_class', 'error_class_extra', 'error', 'output'])


In [8]:
print(" Train Data Sample:")
print(train_data["data"][0])

 Train Data Sample:
{'original_src': 'l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]', 'changed_src': 'l=[]\nwhile True:\n    try:\n        l.append(input())\n    except EOFError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]', 'problem_id': 'p00001', 'original_id': 's196059089', 'changed_id': 's508355022', 'language': 'Python', 'filename_ext': 'py', 'original_status': 'Runtime Error', 'returncode': 1, 'error_class': 'SyntaxError', 'error_class_extra': "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?", 'error': '  File "/home/alex/Documents/research/bug-detection/../input/Project_CodeNet/data/p00001/Python/s196059089.py", line 9\n    print l[i]\n          ^\nSyntaxError: Missing parentheses in call to \'print\'. Did you mean print(l[i])?\n', 'output': ''}


In [9]:
import json
print(json.dumps(train_data["data"][0], indent=4))

{
    "original_src": "l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]",
    "changed_src": "l=[]\nwhile True:\n    try:\n        l.append(input())\n    except EOFError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]",
    "problem_id": "p00001",
    "original_id": "s196059089",
    "changed_id": "s508355022",
    "language": "Python",
    "filename_ext": "py",
    "original_status": "Runtime Error",
    "returncode": 1,
    "error_class": "SyntaxError",
    "error_class_extra": "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?",
    "error": "  File \"/home/alex/Documents/research/bug-detection/../input/Project_CodeNet/data/p00001/Python/s196059089.py\", line 9\n    print l[i]\n          ^\nSyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?\n",
    "output": ""
}


In [10]:

print("Train Data Type:", type(train_data))
print("Test Data Type:", type(test_data))
if "data" in train_data and isinstance(train_data["data"], list):
    print(" 'data' key found in train_data! Length:", len(train_data["data"]))
    print("First Sample Keys:", train_data["data"][0].keys())
else:
    print(" 'data' key missing in train_data!")

if "data" in test_data and isinstance(test_data["data"], list):
    print(" 'data' key found in test_data! Length:", len(test_data["data"]))
    print("First Sample Keys:", test_data["data"][0].keys())
else:
    print(" 'data' key missing in test_data!")

Train Data Type: <class 'dict'>
Test Data Type: <class 'dict'>
 'data' key found in train_data! Length: 43673
First Sample Keys: dict_keys(['original_src', 'changed_src', 'problem_id', 'original_id', 'changed_id', 'language', 'filename_ext', 'original_status', 'returncode', 'error_class', 'error_class_extra', 'error', 'output'])
 'data' key found in test_data! Length: 10919
First Sample Keys: dict_keys(['original_src', 'changed_src', 'problem_id', 'original_id', 'changed_id', 'language', 'filename_ext', 'original_status', 'returncode', 'error_class', 'error_class_extra', 'error', 'output'])


In [11]:
pip install autopep8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00


In [12]:
import ast
import re
import autopep8
import time
import warnings
from tqdm import tqdm

warnings.simplefilter("ignore", SyntaxWarning)

def preprocess_code(code):
    """Cleans and formats Python code while handling errors."""
    if not code:
        return "# Error: Empty Code"

    try:
        code = code.strip()
        code = re.sub(r"#.*", "", code)
        tree = ast.parse(code)
        formatted_code = ast.unparse(tree) if hasattr(ast, "unparse") else code
        final_code = autopep8.fix_code(formatted_code)
        return final_code
    except Exception as e:
        return f"# Error in code: {str(e)}\n{code}"

def process_in_batches(dataset, batch_size=5000):
    """Processes dataset in batches to avoid memory overload."""
    total_samples = len(dataset.get("data", []))
    if total_samples == 0:
        print(" Warning: Empty dataset!")
        return
    for i in tqdm(range(0, total_samples, batch_size), desc="Processing"):
        batch = dataset["data"][i:i + batch_size]
        for sample in batch:
            if "original_src" in sample and isinstance(sample["original_src"], str):
                sample["processed_code"] = preprocess_code(sample["original_src"])
            else:
                sample["processed_code"] = "# Error: Invalid Code Format"
        print(f" Processed {min(i + batch_size, total_samples)} / {total_samples} samples")
        time.sleep(0.2)
process_in_batches(train_data)
process_in_batches(test_data)
print(" Preprocessing completed successfully!")


Processing:  11%|█         | 1/9 [01:04<08:39, 64.99s/it]

 Processed 5000 / 43673 samples


Processing:  22%|██▏       | 2/9 [01:44<05:51, 50.21s/it]

 Processed 10000 / 43673 samples


Processing:  33%|███▎      | 3/9 [02:21<04:24, 44.09s/it]

...

 Processed 10000 / 10919 samples


Processing: 100%|██████████| 3/3 [06:00<00:00, 86.92s/it] 

 Processed 10919 / 10919 samples


Processing: 100%|██████████| 3/3 [06:00<00:00, 120.21s/it]

 Preprocessing completed successfully!


In [13]:
print(len(train_data["data"]))
print(train_data["data"][0]["processed_code"])

43673
# Error in code: Missing parentheses in call to 'print'. Did you mean print(...)? (<unknown>, line 9)
l=[]
while True:
    try:
        l.append(input())
    except SyntaxError:
        break
l.sort(reverse = True)
for i in range(3):
    print l[i]


In [14]:
import json
with open("processed_train_data.json", "w") as f:
    json.dump(train_data, f)
with open("processed_test_data.json", "w") as f:
    json.dump(test_data, f)
print(" Processed data saved!")

 Processed data saved!


In [15]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)
print("CodeBERT Model Loaded!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CodeBERT Model Loaded!


In [17]:
missing_samples = [sample for sample in test_data["data"] if "processed_code" not in sample]
print(f" Missing processed_code samples in test_data: {len(missing_samples)}")

 Missing processed_code samples in test_data: 0


In [18]:
for sample in test_data["data"]:
    if "processed_code" not in sample or not sample["processed_code"]:
        sample["processed_code"] = preprocess_code(sample["original_src"])
print("Missing entries fixed, re-run tokenization.")

Missing entries fixed, re-run tokenization.


In [19]:
train_encodings = tokenizer(
    [sample["processed_code"] for sample in train_data["data"]],
    truncation=True,
    padding="max_length",
    max_length=512
)
test_encodings = tokenizer(
    [sample["processed_code"] for sample in test_data["data"]],
    truncation=True,
    padding="max_length",
    max_length=512
)
print(" Tokenization Done!")

 Tokenization Done!


In [20]:
import torch
class BugFixDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
train_labels = [sample["error_class"] for sample in train_data["data"]]
test_labels = [sample["error_class"] for sample in test_data["data"]]
train_dataset = BugFixDataset(train_encodings, train_labels)
test_dataset = BugFixDataset(test_encodings, test_labels)
print(" Datasets Created Successfully!")

 Datasets Created Successfully!


In [21]:
print(train_data["data"][0].keys())

dict_keys(['original_src', 'changed_src', 'problem_id', 'original_id', 'changed_id', 'language', 'filename_ext', 'original_status', 'returncode', 'error_class', 'error_class_extra', 'error', 'output', 'processed_code'])


In [22]:
label_mapping = {
    "No Error": 0,
    "SyntaxError": 1,
    "TypeError": 2,
    "IndexError": 3,
    "ValueError": 4,
}

In [23]:
import torch
class BugFixDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, label_mapping):
        self.encodings = encodings
        self.labels = [label_mapping.get(label, 0) for label in labels]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [24]:
print(len(train_dataset), len(test_dataset))


43673 10919


In [25]:
train_dataset = BugFixDataset(train_encodings, train_labels, label_mapping)
test_dataset = BugFixDataset(test_encodings, test_labels, label_mapping)
print("Dataset Successfully Created!")


Dataset Successfully Created!


In [26]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(" DataLoaders Ready!")


 DataLoaders Ready!


In [27]:
print(set(train_labels))


{'ZeroDivisionError', 'IndexError', '-11', 'ImportError', '0', 'DeprecationWarning', 'OverflowError', 'UnboundLocalError', 'ValueError', 'NameError', 'KeyError', 'TLEError', 'RecursionError', 'AttributeError', 'SyntaxWarning', 'OSError', 'EOFError', 'SyntaxError', 'TypeError', '2', 'RuntimeError', '255', 'TabError', 'IndentationError', '1', 'FileNotFoundError', 'VisibleDeprecationWarning', 'SparseEfficiencyWarning', 'ModuleNotFoundError'}


In [28]:
from transformers import AutoModel, AutoTokenizer
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
text = "def add(a, b): return a + b"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
print(outputs)


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1635,  0.3595,  0.0261,  ..., -0.2831, -0.2772,  0.3455],
         [-1.0306,  0.1121,  0.5334,  ..., -0.6773, -0.0715,  0.6466],
         [-0.7064,  0.2068, -0.1867,  ..., -0.6525, -0.2080,  0.6185],
         ...,
         [-0.1911,  0.0171, -0.0912,  ...,  0.1607, -0.5508,  0.4795],
         [-0.5272,  0.5221,  0.4231,  ..., -0.7133, -0.6207,  0.9018],
         [-0.1643,  0.3608,  0.0268,  ..., -0.2839, -0.2790,  0.3472]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.4756, -0.3428, -0.5196,  0.1175,  0.3530,  0.0713,  0.4972, -0.3266,
          0.0949, -0.2564,  0.4359,  0.0017, -0.2382,  0.1062,  0.0036,  0.5640,
          0.4242, -0.4990,  0.1064,  0.3437, -0.3256,  0.5128,  0.3463,  0.0351,
         -0.0508,  0.2522,  0.1491,  0.1061,  0.5450,  0.1033,  0.1372,  0.0893,
          0.1787,  0.0130, -0.3124, -0.0641, -0.5261,  0.1648,  0.6857, -0.2942,
         -0.3850,  0.0743,  0.02

In [29]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
model_name = "microsoft/graphcodebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
buggy_code = "def add_numbers(a, b): return a +"
non_buggy_code = "def add_numbers(a, b): return a + b"
inputs_buggy = tokenizer(buggy_code, return_tensors="pt", padding=True, truncation=True)
inputs_non_buggy = tokenizer(non_buggy_code, return_tensors="pt", padding=True, truncation=True)
outputs_buggy = model(**inputs_buggy)
outputs_non_buggy = model(**inputs_non_buggy)
buggy_pred = torch.argmax(outputs_buggy.logits, dim=-1).item()
non_buggy_pred = torch.argmax(outputs_non_buggy.logits, dim=-1).item()

print("Buggy Code Prediction:", "Bug Detected" if buggy_pred == 1 else "No Bug")
print("Non-Buggy Code Prediction:", "Bug Detected" if non_buggy_pred == 1 else "No Bug")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Buggy Code Prediction: Bug Detected
Non-Buggy Code Prediction: Bug Detected


In [30]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
buggy_code = "print(5/)"
input_text = f"Fix: {buggy_code}"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(**inputs, max_length=50)
fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Fixed Code:", fixed_code)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Fixed Code: print(5/(5-5)) print(5/(5-5)) print(5/(5-5)) print(5/(5-5)) print(5/(5-5))


In [31]:
def fix_code(code_snippet):
    input_text = f"fix: {code_snippet}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code


In [32]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
model_name = "huggingface/CodeBERTa-small-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cpu")
buggy_code = "def divide(a, b): return a / b\nprint(divide(10, 0))"
inputs = tokenizer(buggy_code, return_tensors="pt", truncation=True, max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
print("Model Output:", outputs.logits)


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/994k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/483k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at huggingface/CodeBERTa-small-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Output: tensor([[ 0.2277, -0.1095]])


In [34]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cpu")

def detect_bug(code):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)

    probabilities = F.softmax(outputs.logits, dim=1)
    bug_prob = probabilities[0][0].item()
    non_bug_prob = probabilities[0][1].item()

    if bug_prob > non_bug_prob:
        return "Buggy Code Detected!", bug_prob
    else:
        return "Code Seems Fine", non_bug_prob

buggy_code = "def divide(a, b): return a / b\nprint(divide(10, 0))"
status, prob = detect_bug(buggy_code)
print(status, "Confidence:", prob)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Code Seems Fine Confidence: 0.5679237246513367


In [35]:
def fix_bug(code):
    prompt = f"### Buggy Python Code:\n{code}\n### Fixed Python Code:\n"
    inputs = fix_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = fix_model.generate(**inputs, max_length=512)
    fixed_code = fix_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code

In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
fixing_model_name = "Salesforce/codet5-small"
fixing_tokenizer = AutoTokenizer.from_pretrained(fixing_model_name)
fixing_model = AutoModelForSeq2SeqLM.from_pretrained(fixing_model_name).to("cpu")

def fix_code(code):
    inputs = fixing_tokenizer(code, return_tensors="pt", truncation=True, max_length=512).to("cpu")
    outputs = fixing_model.generate(**inputs, max_length=512)
    fixed_code = fixing_tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "b" in fixed_code and "a" not in fixed_code:
        fixed_code = code.replace("/ b", "/ (b if b != 0 else 1)")

    return fixed_code
buggy_code = "def divide(a, b): return a / b\nprint(divide(10, 0))"
fixed_code = fix_code(buggy_code)

print("\n🔹 Fixed Code:\n", fixed_code)


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]


🔹 Fixed Code:
 def divide(a, b): return a / (b if b != 0 else 1)
print(divide(10, 0))


In [37]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00


In [38]:
!pip install transformers accelerate


In [39]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
def detect_and_fix_code(code_input):
    prompt = f"### Buggy Code:\n{code_input}\n\n### Fixed Code:\n"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, do_sample=True, temperature=0.7, top_p=0.9)
    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code.split("### Fixed Code:")[-1].strip()


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

def detect_bug_and_fix(code_input):
    bug_text = f"### Bug Detection:\n{code_input}\n\n### Error Message:\n"
    bug_inputs = tokenizer(bug_text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        bug_outputs = model.generate(**bug_inputs, max_length=128, do_sample=True, temperature=0.6, top_p=0.8)

    error_message = tokenizer.decode(bug_outputs[0], skip_special_tokens=True)
    error_message = error_message.split("### Error Message:")[-1].strip()

    fix_text = f"### Buggy Code:\n{code_input}\n\n### Fixed Code:\n"
    fix_inputs = tokenizer(fix_text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        fix_outputs = model.generate(**fix_inputs, max_length=256, do_sample=True, temperature=0.6, top_p=0.8)

    fixed_code = tokenizer.decode(fix_outputs[0], skip_special_tokens=True)
    fixed_code = fixed_code.split("### Fixed Code:")[-1].strip()

    return error_message, fixed_code

def main():
    with gr.Blocks() as demo:
        gr.Markdown("# Bug Detection & Fixing")
        gr.Markdown("Paste your buggy code below, and get both the error message & the fixed code!")

        with gr.Row():
            code_input = gr.Textbox(lines=10, placeholder="Paste your buggy Python code here...", label="Buggy Code")

        with gr.Row():
            error_output = gr.Textbox(label="Detected Error Message", interactive=False)
            fixed_output = gr.Textbox(label="Fixed Code", interactive=False)

        submit_btn = gr.Button("Detect & Fix Bugs")
        submit_btn.click(detect_bug_and_fix, inputs=[code_input], outputs=[error_output, fixed_output])

    demo.launch()

if __name__ == "__main__":
    main()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ae8ac6f0fc3d9ad44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
